In [1]:
import pandas as pd

In [3]:
price=pd.read_excel(r"./A股1.xlsx",sheet_name="price")
price.head()

,stockcode,简称,tradedate,price
0,000001.SZ,平安银行,2000-01-28,18.50
1,000001.SZ,平安银行,2000-02-29,18.32
2,000001.SZ,平安银行,2000-03-31,18.37
3,000001.SZ,平安银行,2000-04-28,19.06
4,000001.SZ,平安银行,2000-05-31,18.00


In [4]:
pb=pd.read_excel(r"./A股1.xlsx",sheet_name='pb')
pb.head()

,stockcode,简称,tradedate,pb
0,000001.SZ,平安银行,2000-01-28,7.3283
1,000001.SZ,平安银行,2000-02-29,7.257
2,000001.SZ,平安银行,2000-03-31,7.2769
3,000001.SZ,平安银行,2000-04-28,10.1965
4,000001.SZ,平安银行,2000-05-31,9.6294


In [5]:
mkt=pd.read_excel(r"./A股1.xlsx",sheet_name='mkt')
mkt.head()


,stockcode,简称,tradedate,mkt
0,000001.SZ,平安银行,2000-01-28,28709171202
1,000001.SZ,平安银行,2000-02-29,28429838725
2,000001.SZ,平安银行,2000-03-31,28507431080
3,000001.SZ,平安银行,2000-04-28,29578205574
4,000001.SZ,平安银行,2000-05-31,27933247656


In [6]:
def panel_fill(data,time,individual,value):
    data=data.sort_values(by=[time,individual]).reset_index(drop=True)
    if data[value].isna().sum()==0:
        print("原始数据不含有缺失值")
        return data
    else:
        missing_sum=data[value].isna().sum()
        data_group=data.groupby(individual)
        data=data_group.transform(lambda x: x.interpolate(method='linear',limit_direction='both')).join(data.reset_index()[individual])
        if data[value].isna().sum()==0:
            print("原始数据缺失值数量为：",missing_sum,"，已处理")
            return data
        else:
            data=data[[time,individual]].join(data.groupby(time)[value].transform(lambda x: x.fillna(x.mean())).reset_index(drop=True))
            if data[value].isna().sum()==0:
                print("存在有组全部缺失，已处理")
                return data
            else:
                print("经处理后，仍含有缺失值")
                return data

In [7]:
pb['pb']=pd.to_numeric( pb['pb'],errors='coerce')
BM=pb.copy()
BM['BM']=1/BM['pb']
BM=panel_fill(BM,"tradedate",'stockcode','pb')

原始数据缺失值数量为： 129 ，已处理


In [8]:
pb['tradedate'] = pd.to_datetime(pb.tradedate)
pb.dtypes

stockcode            object
简称                   object
tradedate    datetime64[ns]
pb                  float64
dtype: object

In [9]:
ipodate = pd.read_excel(r"./上市日期.xlsx")
ipodate.head()

,stockcode,证券简称,ipodate
0,000005.SZ,ST星源,1990-12-10
1,600651.SH,飞乐音响,1990-12-19
2,600653.SH,申华控股,1990-12-19
3,600654.SH,ST中安,1990-12-19
4,600602.SH,云赛智联,1990-12-19


In [10]:
price['tradedate'] = pd.to_datetime(price.tradedate)
#ST['entry_dt'] = pd.to_datetime(ST.entry_dt)
#ST['remove_dt'] = pd.to_datetime(ST.remove_dt)
mkt['tradedate'] = pd.to_datetime(mkt.tradedate)
pb['tradedate'] = pd.to_datetime(pb.tradedate)
ipodate['ipodate'] = pd.to_datetime(ipodate.ipodate)

In [11]:
ipodate = pd.read_excel(r"D:\研究生毕设\毕设\value premium\数据\上市日期.xlsx")
ipodate.head()

,stockcode,证券简称,ipodate
0,000005.SZ,ST星源,1990-12-10
1,600651.SH,飞乐音响,1990-12-19
2,600653.SH,申华控股,1990-12-19
3,600654.SH,ST中安,1990-12-19
4,600602.SH,云赛智联,1990-12-19


In [12]:
pb1 =pb.drop(pb[(pb['简称'].str.contains('ST') )|( pb['简称'].str.contains('ST*'))].index)

In [13]:
BM1 =BM.drop(BM[(BM['简称'].str.contains('ST') )|(BM['简称'].str.contains('ST*'))].index)

In [14]:
mkt1 =mkt.drop(mkt[(mkt['简称'].str.contains('ST') )|( mkt['简称'].str.contains('ST*'))].index)

In [15]:
price1 =price.drop(price[(price['简称'].str.contains('ST') )|( price['简称'].str.contains('ST*'))].index)

In [16]:
price1['tradedate'] = pd.to_datetime(price1.tradedate)
mkt1['tradedate'] = pd.to_datetime(mkt1.tradedate)
pb1['tradedate'] = pd.to_datetime(pb1.tradedate)
BM1['tradedate'] = pd.to_datetime(BM1.tradedate)

In [17]:
f=pd.merge(BM1,mkt1,left_on=['tradedate',"stockcode"],right_on=['tradedate',"stockcode"])
f=f.sort_values(by=['tradedate',"stockcode"]).reset_index(drop=True)

In [18]:
f.head()

,简称_x,tradedate,pb,BM,stockcode,简称_y,mkt
0,平安银行,2000-01-28,7.3283,0.136457,000001.SZ,平安银行,28709171202
1,万科A,2000-01-28,3.2113,0.311400,000002.SZ,万科A,6656753978
2,国华网安,2000-01-28,16.0083,0.062468,000004.SZ,国华网安,1125287566
3,深振业A,2000-01-28,2.4339,0.410863,000006.SZ,深振业A,2786972025
4,神州高铁,2000-01-28,68.7030,0.014555,000008.SZ,神州高铁,6702441928


In [19]:
f['tradedate'] = pd.to_datetime(f.tradedate)
f.dtypes
ipodate['ipodate'] = pd.to_datetime(ipodate.ipodate)

In [20]:
import datetime

In [21]:
#先计算上市一年后的日期，然后用交易日和该日期进行比较，只保留上市一年后的数据
ipodate['entrydate']=ipodate['ipodate']+datetime.timedelta(365)

#将其合并进因子数据（左连接）
f=pd.merge(f,ipodate,how='left',left_on='stockcode',right_on='stockcode')

#选取数据
f=f.loc[f.tradedate>=f.entrydate].reset_index(drop=True)


In [22]:
f

,简称_x,tradedate,pb,BM,stockcode,简称_y,mkt,证券简称,ipodate,entrydate
0,平安银行,2000-01-28,7.3283,0.136457,000001.SZ,平安银行,28709171202,平安银行,1991-04-03,1992-04-02
1,万科A,2000-01-28,3.2113,0.311400,000002.SZ,万科A,6656753978,万科A,1991-01-29,1992-01-29
2,国华网安,2000-01-28,16.0083,0.062468,000004.SZ,国华网安,1125287566,国华网安,1991-01-14,1992-01-14
3,深振业A,2000-01-28,2.4339,0.410863,000006.SZ,深振业A,2786972025,深振业A,1992-04-27,1993-04-27
4,神州高铁,2000-01-28,68.7030,0.014555,000008.SZ,神州高铁,6702441928,神州高铁,1992-05-07,1993-05-07
...,...,...,...,...,...,...,...,...,...,...
417229,明微电子,2022-03-30,6.8737,0.145482,688699.SH,明微电子,11280138240,明微电子,2020-12-18,2021-12-18
417230,中控技术,2022-03-30,8.1114,0.123283,688777.SH,中控技术,34345373990,中控技术,2020-11-24,2021-11-24
417231,科思科技,2022-03-30,1.9716,0.507202,688788.SH,科思科技,5881095304,科思科技,2020-10-22,2021-10-22
417232,天能股份,2022-03-30,2.5796,0.387657,688819.SH,天能股份,30883617000,天能股份,2021-01-18,2022-01-18


In [23]:
f.drop(['ipodate','entrydate'],axis=1,inplace=True)

In [24]:
f

,简称_x,tradedate,pb,BM,stockcode,简称_y,mkt,证券简称
0,平安银行,2000-01-28,7.3283,0.136457,000001.SZ,平安银行,28709171202,平安银行
1,万科A,2000-01-28,3.2113,0.311400,000002.SZ,万科A,6656753978,万科A
2,国华网安,2000-01-28,16.0083,0.062468,000004.SZ,国华网安,1125287566,国华网安
3,深振业A,2000-01-28,2.4339,0.410863,000006.SZ,深振业A,2786972025,深振业A
4,神州高铁,2000-01-28,68.7030,0.014555,000008.SZ,神州高铁,6702441928,神州高铁
...,...,...,...,...,...,...,...,...
417229,明微电子,2022-03-30,6.8737,0.145482,688699.SH,明微电子,11280138240,明微电子
417230,中控技术,2022-03-30,8.1114,0.123283,688777.SH,中控技术,34345373990,中控技术
417231,科思科技,2022-03-30,1.9716,0.507202,688788.SH,科思科技,5881095304,科思科技
417232,天能股份,2022-03-30,2.5796,0.387657,688819.SH,天能股份,30883617000,天能股份


In [106]:
f['ym']=f.tradedate.apply(lambda x : x.year*100 + x.month)
f.head()

,简称_x,tradedate,pb,BM,stockcode,简称_y,mkt,证券简称,ym
0,平安银行,2000-01-28,7.3283,0.136457,000001.SZ,平安银行,28709171202,平安银行,200001
1,万科A,2000-01-28,3.2113,0.311400,000002.SZ,万科A,6656753978,万科A,200001
2,国华网安,2000-01-28,16.0083,0.062468,000004.SZ,国华网安,1125287566,国华网安,200001
3,深振业A,2000-01-28,2.4339,0.410863,000006.SZ,深振业A,2786972025,深振业A,200001
4,神州高铁,2000-01-28,68.7030,0.014555,000008.SZ,神州高铁,6702441928,神州高铁,200001
